In [1]:
import pygame
import sys
import random
import math

from pygame.locals import *

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard
from keras.optimizers import Adam

import time 
import os
from tqdm import tqdm

from collections import deque

import numpy as np


C:\Users\Eric\Anaconda3\envs\tf_env\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Eric\Anaconda3\envs\tf_env\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Eric\Anaconda3\envs\tf_env\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Eric\Anaconda3\envs\tf_env\lib\site-packages\tensorflow\python\f

In [24]:
# # setup game
# mainClock = pygame.time.Clock()


# create window
WINDOWWIDTH = 800
WINDOWHEIGHT = 800


# resources
BRICK = (255, 0, 0)
ORE = (190, 190, 190)
WHEAT = (255, 215, 0)
SHEEP = (124, 252, 0)
WOOD = (0, 100, 0)
DESSERT = (139, 69, 19)

resources = {BRICK: 'Brick', ORE: 'Ore', WHEAT : 'Wheat',
            SHEEP : 'Sheep', WOOD:'Wood', DESSERT:'Dessert'}

# players
P1 = (0, 0, 255)
P2 = (255, 20, 147)
P3 = (255, 165, 0)
P4 = (70, 130, 180)

# hex locations
hexTiles = [(0,0), (0,1), (0,-1), (-1,0.5), (-1,-0.5), (1,0.5), (1,-0.5), (2,0), (-2,0), (0,2),
                (0,-2), (1,1.5), (1,-1.5), (-1,1.5), (-1, -1.5), (2,1), (2,-1), (-2,1), (-2,-1)]

hexProbabilities = [5, 2, 6, 3, 8, 10, 9, 12, 11, 4, 8, 10, 9, 4, 5, 6, 3, 11]


probabilities_values = {
    2 : 1,
    3 : 2,
    4 : 3,
    5 : 4,
    6 : 5,
    7 : 0,
    8 : 5,
    9 : 4,
    10 : 3,
    11 : 2,
    12 : 1,
}

resource_values = {
    'Brick' : 1,
    'Ore' : 3,
    'Wheat' : 3,
    'Sheep' : 2,
    'Wood' : 2,
    'Dessert' : 0
}

position_names = {
 '315395': 'AA',
 '355325': 'AB',
 '435325': 'AC',
 '475395': 'AD',
 '435455': 'AE',
 '355455': 'AF',
 '315525': 'AG',
 '475525': 'AH',
 '435595': 'AI',
 '355595': 'AJ',
 '315255': 'AK',
 '355185': 'AL',
 '435185': 'AM',
 '475255': 'AN',
 '195455': 'AO',
 '235395': 'AP',
 '235525': 'AQ',
 '195325': 'AR',
 '235255': 'AS',
 '555395': 'AT',
 '595455': 'AU',
 '555525': 'AV',
 '555255': 'AW',
 '595325': 'AX',
 '675325': 'AY',
 '715395': 'AZ',
 '675455': 'BA',
 '75395':  'BB',
 '115325': 'BC',
 '115455': 'BD',
 '315665': 'BE',
 '475665': 'BF',
 '435735': 'BG',
 '355735': 'BH',
 '315115': 'BI',
 '35545':  'BJ',
 '43545':  'BK',
 '475115': 'BL',
 '595595': 'BM',
 '555665': 'BN',
 '555115': 'BO',
 '595185': 'BP',
 '195595': 'BQ',
 '235665': 'BR',
 '195185': 'BS',
 '235115': 'BT',
 '715525': 'BU',
 '675595': 'BV',
 '675185': 'BW',
 '715255': 'BX',
 '75525':  'BY',
 '115595': 'BZ',
 '75255':  'CA',
 '115185': 'CB'}


FIRSTPLACE_REWARD = 200
SECONDPLACE_REWARD = 100
THIRDPLACE_REWARD = 0
FOURTHPLACE_REWARD = -200

REPLAY_MEMORY_SIZE = 50_000 # How many last steps to keep for model training
MIN_REPLAY_MEMORY_SIZE = 1_000 # Minimum number of steps in a memory to start training
MODEL_NAME = "256x2"


MINIBATCH_SIZE = 64 # How many steps (samples) to use for training
DISCOUNT = 0.99
UPDATE_TARGET_EVERY = 5  # Terminal states (end of episodes)



In [25]:
# Own Tensorboard class
class ModifiedTensorBoard(TensorBoard):

    # Overriding init to set initial step and writer (we want one log file for all .fit() calls)
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.step = 1
        self.writer = tf.summary.FileWriter(self.log_dir)

    # Overriding this method to stop creating default log writer
    def set_model(self, model):
        pass

    # Overrided, saves logs with our step number
    # (otherwise every .fit() will start writing from 0th step)
    def on_epoch_end(self, epoch, logs=None):
        self.update_stats(**logs)

    # Overrided
    # We train for one batch only, no need to save anything at epoch end
    def on_batch_end(self, batch, logs=None):
        pass

    # Overrided, so won't close writer
    def on_train_end(self, _):
        pass

    # Custom method for saving own metrics
    # Creates writer, writes custom metrics and closes writer
    def update_stats(self, **stats):
        self._write_logs(stats, self.step)



class DQNAgent:
    def __init__(self):
        
        # main model -- this is what gets trained after every step
        self.model = self.create_model()
        
        # target model -- this is what we .predict against every set
        self.target_model = self.create_model()
        self.target_model.set_weights(self.model.get_weights())
        
        # how we keep the agent taking consistant steps
        self.replay_memory = deque(maxlen=REPLAY_MEMORY_SIZE)
        
        self.tensorboard = ModifiedTensorBoard(log_dir = "logs/{}-{}".format(MODEL_NAME, int(time.time())))
        
        self.target_update_counter = 0
        
        
    def create_model(self):
        # conv net
        model = Sequential()
        model.add(Conv2D(256, (3,3), input_shape = env.OBSERVATION_SPACE_VALUES))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(2,2))
        model.add(Dropout(0.2))
        
        model.add(Conv2D(256, (3,3)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(2,2))
        model.add(Dropout(0.2))
        
        model.add(Flatten())
        model.add(Dense(64))
        
        model.add(Dense(env.ACTION_SPACE_SIZE, activation = 'linear'))
        model.compile(loss="mse", optimizer=Adam(lr=0.001), metrics=['accuracy'])
        
        return model
    
    def update_replay_memory(self, transition):
        self.replay_memory.append(transition)
        
    def get_qs(self, state):
        return self.model.predict(np.array(state).reshape(-1, *state.shape)/255)[0]

    
    def train(self, terminal_state, step):
        if len(self.replay_memory) < MIN_REPLAY_MEMORY_SIZE:
            return
        
        minibatch = random.sample(self.replay_memory, MINIBATCH_SIZE)
        
        current_states = np.array([transition[0] for transition in minibatch])/255
        current_qs_list = self.model.predict(current_states)
        
        new_current_states = np.array([transition[3] for transition in minibatch])/255
        future_qs_list = self.target_model.predict(new_current_states)
        
        X = []
        y = []
        
        # use to calculate the qs
        for index, (current_state, action, reward, new_current_state, done) in enumerate(minibatch):
            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = reward + DISCOUNT * max_future_q
            else:
                new_q = reward
                
            current_qs = current_qs_list[index]
            current_qs[action] = new_q
            
            X.append(current_state)
            y.append(current_qs)
            
        self.model.fit(np.array(X)/255, np.array(y), batch_size = MINIBATCH_SIZE, verbose = 0, 
                       shuffle = False, callbacks =[self.tensorboard] if terminal_state else None)

        # updating to determine if we want to update target_model yet
        if terminal_state:
            self.target_update_counter += 1
            
        if self.target_update_counter > UPDATE_TARGET_EVERY:
            self.target_model.set_weights(self.model.get_weights())
            self.target_update_counter = 0


In [33]:
# Board Class
class Board:
    hexes = []
    settlements = []
    instructions = ""
    
    def __init__(self, size, screen, zipHex, settlementLoc):
        self.screen = screen
        self.cX = self.screen.get_rect().centerx
        self.cY = self.screen.get_rect().centery
        
        self.hexHeight = 0.866 * size
        
        # the hexs
        self.hexes=[]
        for tile in zip(hexTiles, zipHex):
            self.hexes.append(Hex(tile[0], size, tile[1][0], tile[1][1], self, settlementLoc))
        
        # set up the settlement locations
        self.settlements = settlementLoc 

            
    def draw(self):
        # draw the HEXes
        for h in self.hexes:
            h.draw()

        # draw the settlement locations
        for item in self.settlements:
            item.draw()

        # message to user
        basicFont = pygame.font.SysFont(None, 24)
        text = basicFont.render("HEllo", True, (255,255,255), (0,0,0))
        textRect = text.get_rect()
        textRect.left = 5
        textRect.top = 560
        self.screen.blit(text, textRect)
        
        
        
""" Create a settlement class"""
class Settlement:
    def __init__(self, location, tileOwner):
        self.location = location
        self.locationx = location[0]
        self.locationy = location[1]
        self.id = position_names[str(location[0])+str(location[1])]
        self.onTiles = [tileOwner]
        self.neighbours = [] # do I need?
        self.city = False
        self.owner = -1
        self.check_coords = [
            str(self.locationx + 80)+ str(self.locationy),
            str(self.locationx - 80)+ str(self.locationy),
            str(self.locationx + 40)+ str(self.locationy + 70),
            str(self.locationx + 40)+ str(self.locationy - 70),
            str(self.locationx - 40)+ str(self.locationy + 70),
            str(self.locationx - 40)+ str(self.locationy - 70),
            str(self.locationx + 40)+ str(self.locationy + 60),
            str(self.locationx + 40)+ str(self.locationy - 60),
            str(self.locationx - 40)+ str(self.locationy + 60),
            str(self.locationx - 40)+ str(self.locationy - 60),

        ]
        
        self.omit_list = []
        for i in self.check_coords:
            try:
                self.omit_list.append(position_names[i])
            except:
                pass


        
    def draw(self):
#         pygame.draw.rect(screen, (0,0,0), (self.locationx, self.locationy, 20, 20),2)
        

        font = pygame.font.Font(None, 20)
        text = font.render(str(self.id), 1, (255,255,255), (0,0,0))
        textpos = text.get_rect()
        textpos.left = self.locationx + 3
        textpos.top = self.locationy + 3
        screen.blit(text, textpos)


        
# Hex Class
class Hex:

    def __init__(self, centre, size, resource, probability, board, settlementLoc):
        self.centre = centre
        self.hexX = board.cX +(1.5*centre[0]*size)
        self.hexY = board.cY+(2*centre[1]*board.hexHeight)
        self.hexSize = size
        self.resource = resource
        self.value = probability	# chance of getting this resource
        self.hasRobber = False
        
        # calculate each 6 points so we only do it once
        self.points = []
        self.points.append((self.hexX - self.hexSize, self.hexY))
        self.points.append((self.hexX - (self.hexSize/2), self.hexY - (0.866*self.hexSize)))
        self.points.append((self.hexX + (self.hexSize/2), self.hexY - (0.866*self.hexSize)))
        self.points.append((self.hexX + self.hexSize, self.hexY))
        self.points.append((self.hexX + (self.hexSize/2), self.hexY + (0.866*self.hexSize)))
        self.points.append((self.hexX - (self.hexSize/2), self.hexY + (0.866*self.hexSize)))
        
        # add the points of all settlements to the settlement lists
#         global settlementLoc
        newSets = [] # store all the locations of the new settlements from this til
        newSets += [((10*math.floor(X / 10))-5, (10*math.floor(Y / 10))-5) for (X,Y) in self.points]
        
        for ns in newSets: # these are the ones for the current hex
            for s in settlementLoc: # these are the ones that have been established
                if s.location == ns:  # so if currently is an established settlement
                    newSets.remove(ns)  # remove it from the set
                    s.onTiles.append(self)  # add to that settlement's tile owneres
                    break
                    
        for ns in newSets:	# these are the remainders
            settlementLoc.append(Settlement(ns, self))
        
    def draw(self): 
        # draw the hex
        pygame.draw.polygon(screen, self.resource, self.points, 0)
        
        # draw the text
        if self.value > 0:
            font = pygame.font.Font(None, 36)
            text = font.render(str(self.value), 1, (0,0,0))
            textpos = text.get_rect()
            textpos.left = self.hexX - 5
            textpos.top = self.hexY - 10
            screen.blit(text, textpos)

class Player:
    def __init__(self, turn, bot):
        self.turn = turn
        self.value = 0
        self.bot = True
        
    def action(self, choice, choices, settlers, pos_values):
        
        # if it's a bot, then selects random option from possible position
        if self.bot:
            choice = random.choice(choices)
        else:
            choice = choice
            
            
        # removes choice from possible choices
        choices.remove(choice)
        
        # removes adjacent choices from possible choices
        for sett in settlers.settlements:
            if sett.id == choice:
                try:
                    [choices.remove(i) for i in sett.omit_list]
                except:
                    pass
                break
                
        # updates player value
        self.value += pos_values[choice]
#         print('Player {0} has {1} points'.format(self.turn, self.value))
        
        return self.value
    
    
class CatanEnv:
    SIZE = 10
    OBSERVATION_SPACE_VALUES = (SIZE, SIZE, 3)
    ACTION_SPACE_SIZE = len(choices)

    def setup(self):
        
        settlementLoc = []
        screen = pygame.display.set_mode((WINDOWWIDTH,WINDOWHEIGHT), 0, 32)

        screen.fill((0,194,249))

        pygame.display.set_caption('Catan')
        
        random.shuffle(hexProbabilities)

        hexResources = [BRICK, BRICK, BRICK, ORE, ORE, ORE, WHEAT, WHEAT, WHEAT, WHEAT, SHEEP, 
                        SHEEP, SHEEP, SHEEP, WOOD, WOOD, WOOD, WOOD]
        
        zipHex = []

        zipHex = list(zip(hexResources, hexProbabilities))

        zipHex.append((DESSERT, 7))


        random.shuffle(zipHex)


        
        self.player1 = Player(1, bot=False)
        self.player2 = Player(2, bot=True)
        self.player3 = Player(3, bot=True)
        self.player4 = Player(4, bot=True)

        pos_values = {}

        turn = 1
        done = False
        settlers = Board(80, screen, zipHex, settlementLoc)

        
        for position in settlers.settlements:
            for tile in position.onTiles:
                try:
                    pos_values[position.id] += probabilities_values[tile.value] * resource_values[str(resources[tile.resource])] 
#                     print(tile.value)
                except:
                    pos_values[position.id] = probabilities_values[tile.value] * resource_values[str(resources[tile.resource])] 
                    
        choices = [k for k, _ in pos_values.items()]
        
        
        
        
        return turn, done, choices, settlers, pos_values, screen
        
        
    def step(self, turn, choice, choices, settlers, pos_values, done):
        if turn == 1:
            self.player1.action(choice, choices, settlers, pos_values)
            turn += 1
        
        elif turn == 2:
            self.player2.action(choice, choices, settlers, pos_values)
            turn += 1
            
        elif turn == 3:
            self.player3.action(choice, choices, settlers, pos_values)
            turn += 1
            
        elif turn == 4:
            self.player4.action(choice, choices, settlers, pos_values)
            turn += 1

        elif turn == 5:
            self.player4.action(choice, choices, settlers, pos_values)
            turn += 1

        elif turn == 6:
            self.player3.action(choice, choices, settlers, pos_values)
            turn += 1

        elif turn == 7:
            self.player2.action(choice, choices, settlers, pos_values)
            turn += 1

        elif turn == 8:
            self.player1.action(choice, choices, settlers, pos_values)
            turn += 1
        
        else:
            #ends the game
            done = True
            pass
        
        player_values = [{'player':1, 'score':self.player1.value},
                         {'player':2, 'score':self.player2.value},
                         {'player':3, 'score':self.player3.value},
                         {'player':4, 'score':self.player4.value}]
        
        return turn, done, player_values
    
    
    def render(self, screen):
        # draw the board        
        settlers.draw()

        # draw the window onto the screen
        pygame.display.update()

        



In [34]:
# Create models folder
if not os.path.isdir('catan_models'):
    os.makedirs('catan_models')


env = CatanEnv()
choice = ''

ep_rewards = [-200]

agent = DQNAgent()

EPISODES = 1_000
epsilon = 1
EPSILON_DECAY = 0.99975
MIN_EPSILON = 0.001

MIN_REWARD = 0
AGGREGATE_STATS_EVERY = int(EPISODES/10)

for episode in tqdm(range(1, EPISODES + 1), ascii = True, unit="episodes"):
    agent.tensorboard.step = episode
    t, d, choices, settlers, pos_values, screen = env.setup()
    current_state = [t, choices, settlers, pos_values, screen]

    while not d:
        t, d, pv = env.step(t,choice, choices, settlers, pos_values, d)
        
        if np.random.random() > epsilon:
            choice = np.argmax(agent.get_qs(current_state))            




    # getting the ranks and assigned rewards based on ranks
    def myFunc(e):
        return e['score']
    pv.sort(key=myFunc, reverse=True)
    prev = None
    for index, item in enumerate(pv):
        if item['score'] != prev:
            item['rank']=index+1
        else:
            item['rank']=index
        prev = item['score']

        if item['rank']==1:
            item['reward'] = FIRSTPLACE_REWARD
        elif item['rank']==2:
            item['reward'] = SECONDPLACE_REWARD
        elif item['rank']==3:
            item['reward'] = THIRDPLACE_REWARD
        elif item['rank']==4:
            item['reward'] = FOURTHPLACE_REWARD

    for i in pv:
        if i['player']==1:
            reward = i['reward']
            print(i)

    new_state = [t, choices, settlers, pos_values, screen]

    
    agent.update_replay_memory((current_state, choice, reward, new_state, d))
    agent.train(d, t)

    
    # Append episode reward to a list and log stats (every given number of episodes)
    ep_rewards.append(reward)
    if not episode % AGGREGATE_STATS_EVERY or episode == 1:
        average_reward = sum(ep_rewards[-AGGREGATE_STATS_EVERY:])/len(ep_rewards[-AGGREGATE_STATS_EVERY:])
        agent.tensorboard.update_stats(reward_avg=average_reward, epsilon=epsilon)
#         min_reward = min(ep_rewards[-AGGREGATE_STATS_EVERY:])
#         max_reward = max(ep_rewards[-AGGREGATE_STATS_EVERY:])
#         agent.tensorboard.update_stats(reward_avg=average_reward, reward_min=min_reward, reward_max=max_reward, epsilon=epsilon)
        


        # Save model, but only when min reward is greater or equal a set value
        if min_reward >= MIN_REWARD:
            agent.model.save(f'models/{MODEL_NAME}__{max_reward:_>7.2f}max_{average_reward:_>7.2f}avg_{min_reward:_>7.2f}min__{int(time.time())}.model')

    # Decay epsilon
    if epsilon > MIN_EPSILON:
        epsilon *= EPSILON_DECAY
        epsilon = max(MIN_EPSILON, epsilon)

    


  0%|                                           | 0/1000 [00:00<?, ?episodes/s]

{'player': 1, 'score': 14, 'rank': 4, 'reward': -200}


  1%|3                                 | 10/1000 [00:00<00:56, 17.67episodes/s]

{'player': 1, 'score': 26, 'rank': 2, 'reward': 100}
{'player': 1, 'score': 24, 'rank': 3, 'reward': 0}
{'player': 1, 'score': 37, 'rank': 1, 'reward': 200}
{'player': 1, 'score': 2, 'rank': 4, 'reward': -200}
{'player': 1, 'score': 35, 'rank': 2, 'reward': 100}
{'player': 1, 'score': 49, 'rank': 1, 'reward': 200}
{'player': 1, 'score': 49, 'rank': 1, 'reward': 200}
{'player': 1, 'score': 36, 'rank': 2, 'reward': 100}
{'player': 1, 'score': 9, 'rank': 4, 'reward': -200}


AttributeError: 'list' object has no attribute 'shape'

In [37]:
pos_values

{'AA': 22,
 'AB': 22,
 'AC': 21,
 'AD': 21,
 'AE': 23,
 'AF': 23,
 'AG': 13,
 'AH': 19,
 'AI': 17,
 'AJ': 11,
 'AK': 16,
 'AL': 17,
 'AM': 26,
 'AN': 24,
 'AO': 20,
 'AP': 18,
 'AQ': 14,
 'AR': 18,
 'AS': 16,
 'AT': 9,
 'AU': 21,
 'AV': 29,
 'AW': 24,
 'AX': 9,
 'AY': 6,
 'AZ': 0,
 'BA': 15,
 'BB': 8,
 'BC': 14,
 'BD': 14,
 'BE': 6,
 'BF': 12,
 'BG': 4,
 'BH': 4,
 'BI': 11,
 'BJ': 5,
 'BK': 5,
 'BL': 20,
 'BM': 23,
 'BN': 8,
 'BO': 15,
 'BP': 21,
 'BQ': 8,
 'BR': 2,
 'BS': 12,
 'BT': 6,
 'BU': 15,
 'BV': 15,
 'BW': 6,
 'BX': 6,
 'BY': 6,
 'BZ': 6,
 'CA': 6,
 'CB': 6}

In [ ]:
# """ Set up the information for the settlements """
# settlers = Board(80)


# pos_values = {}

# for position in settlers.settlements:
#     for tile in position.onTiles:
#         try:
#             pos_values[position.id] += probabilities_values[tile.value] * resource_values[str(resources[tile.resource])] 
#         except:
#             pos_values[position.id] = probabilities_values[tile.value] * resource_values[str(resources[tile.resource])] 
        
# choices = [k for k, _ in pos_values.items()]

# # {k: v for k, v in sorted(pos_values.items(), key=lambda item: item[1])}


In [ ]:

# run the game loop
pygame.init()
p1 = Player(1, bot=False)
p2 = Player(2, bot=True)
p3 = Player(3, bot=True)
p4 = Player(4, bot=True)

while not done:
    # check for the QUIT event
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            sys.exit()
            
            
        elif event.type == MOUSEBUTTONDOWN:
#         else:
            # player 1
            choice = random.choice(choices)
            print("choice = {}".format(choice))
            choices.remove(choice)
            for sett in settlers.settlements:
                if sett.id == choice:
                    # A transparent surface with per-pixel alpha.
                    circle = pygame.Surface((60, 60), pygame.SRCALPHA)
                    # Draw a circle onto the `circle` surface.
                    pygame.draw.circle(circle, P1, [20, 20], 30)                    
                    screen.blit(circle, [sett.locationx, sett.locationy])
                    try:
                        [choices.remove(i) for i in sett.omit_list]
                    except:
                        pass
                    break
            
            p1.action(choice)
            
            # player 2
            choice = random.choice(choices)
            print("choice = {}".format(choice))
            choices.remove(choice)
            for sett in settlers.settlements:
                if sett.id == choice:

                    try:
                        [choices.remove(i) for i in sett.omit_list]
                    except:
                        pass
                    break
            
            p2.action(choice)
            
            # player 3
            choice = random.choice(choices)
            print("choice = {}".format(choice))
            choices.remove(choice)
            for sett in settlers.settlements:
                if sett.id == choice:

                    try:
                        [choices.remove(i) for i in sett.omit_list]
                    except:
                        pass
                    break
            
            p3.action(choice)
            
            # player 4
            choice = random.choice(choices)
            print("choice = {}".format(choice))
            choices.remove(choice)
            for sett in settlers.settlements:
                if sett.id == choice:

                    try:
                        [choices.remove(i) for i in sett.omit_list]
                    except:
                        pass
                    break
            
            p4.action(choice)

            done = True
            
#         elif event.type == MOUSEBUTTONDOWN:
#             click = pygame.mouse.get_pos()
#             x = (10*math.floor(click[0] / 10))-5
#             y = (10*math.floor(click[1] / 10))-5       
           
#             for sett in settlers.settlements:
#                 if sett.location == (x,y):
#                     choice = sett.id
#                     p1.action(choice)
#                     clicks -= 1

#                     for tile in sett.onTiles:
#                         print (str(resources[tile.resource]) + " is " + str(tile.value))

        
    # draw the board        
    settlers.draw()

    # draw the window onto the screen
    pygame.display.update()
    mainClock.tick(40)

pygame.quit()
